In [ ]:
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List, Literal
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
import operator
from pydantic import BaseModel, Field
from enum import StrEnum
from langchain.agents import create_agent
from langchain.agents.structured_output import ProviderStrategy

_ = load_dotenv()

In [41]:
class EmailSchema(BaseModel):
    """Schema for representing an email message."""
    id: Annotated[str, Field(description="The unique identifier of the email")]
    receivedAt: Annotated[str, Field(description="The timestamp when the email was received")]
    sender: Annotated[str, Field(description="The sender of the email")]
    subject: Annotated[str, Field(description="The subject of the email")]
    body: Annotated[str, Field(description="The body content of the email")]
    reciever: Annotated[str, Field(description="The receiver of the email")]

class CustomerInformationSchema(BaseModel):
    """Schema for extracting customer information from the email."""
    name: Annotated[str, Field(description="The name of the customer.")]
    dates: Annotated[List[str], Field(description="The dates referenced in the email by the customer, in the format YYYY-MM-DD.")]
    amounts: Annotated[List[str], Field(description="The amounts referenced in the email by the customer.")]
    invoice_references: Annotated[List[str], Field(description="The list of invoice IDs referenced in the email by the customer.")]
    dispute_details: Annotated[str, Field(description="The details of the dispute as described by the customer in the email.")]

class Category(StrEnum):
    PAYMENT_CLAIM = "Payment Claim"
    DISPUTE = "Dispute"
    GENERAL_AR_REQUEST = "General AR Request"


In [ ]:
class AgentExpectedOutput(BaseModel):
    """Schema for representing the expected output of the agent.
    First the agent is expected to categorize the email into one of the three categories: Payment Claim, Dispute, or General AR Request.
    Then, the agent is expected to generate a response email based on the input email and the category, and also extract the customer information from the email.
    """
    response_email: Annotated[EmailSchema, Field(description=(
        "The response email that the agent should generate based on the input email and the category. "
        "Our company's email address is: info@transformance.com. "
        "if the category is Payment Claim, the response email should include a payment claim response. "
        "if the category is Dispute, the response email should include a dispute response. "
        "if the category is General AR Request, the response email should include a general AR request response."
    ))]
    category: Annotated[Category, Field(description="The category of the email, which can be one of the following: Payment Claim, Dispute, General AR Request.")]
    customer_information: Annotated[CustomerInformationSchema, Field(description="The extracted customer information from the email, which includes the customer's name, dates, amounts, invoice references, and dispute details if applicable.")]


email_agent = create_agent(
    model=ChatOpenAI(model="gpt-4.1-mini-2025-04-14", temperature=0.0),
    response_format=ProviderStrategy(AgentExpectedOutput),
    system_prompt=SystemMessage(
        content=(
            "You are an assistant for categorizing and responding to emails. Also you should extract relevant customer information from the email. "
            "Your task is to read the input email, categorize it into one of the following categories: Payment Claim, Dispute, or General AR Request, and then generate a response email based on the input email and the category. "
            "Additionally, you should extract relevant customer information from the email, including the customer's name, dates, amounts, invoice references, and dispute details if applicable."
        )
    )
)

In [33]:
import json

with open("data/Sample Emails.json", "r") as f:
    email_data = json.load(f).get("emails", [])

sample_emails = [json.dumps(email) for email in email_data[:20]]

In [36]:
responses = await email_agent.abatch(
    [{"messages": [HumanMessage(content=sample_email)]} for sample_email in sample_emails]
)

In [37]:
results = [
    AgentExpectedOutput(
        **json.loads(response.get("messages", [])[-1].content)
    )
    for response in responses
]

In [40]:
results[7]

AgentExpectedOutput(response_email=EmailSchema(id='response_email_008', receivedAt='2026-01-20T09:00:00Z', sender='info@transformance.com', subject='Re: Statement of account request INV-10007', body='Dear UrbanFoods Inc Finance Team,\n\nThank you for your request. We will prepare and send you the latest statement of account including the details for invoice INV-10007 shortly.\n\nIf you have any other questions or need further assistance, please do not hesitate to contact us.\n\nBest regards,\nTransformance Accounts Receivable Team', reciever='ap@urbanfoods.com'), category=<Category.GENERAL_AR_REQUEST: 'General AR Request'>, customer_information=CustomerInformationSchema(name='UrbanFoods Inc Finance Team', dates=[], amounts=[], invoice_references=['INV-10007'], dispute_details=''))